Notebook to test forecast chart when units are changed from years to months

In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

from pof.component import Component
from pof.failure_mode import FailureMode
from pof.loader.asset_model_loader import AssetModelLoader
from pof.paths import Paths
from pof.data.asset_data import SimpleFleet
from pof.units import scale_units, unit_ratio

In [2]:
# Forecast years
START_YEAR = 2015
END_YEAR = 2024
CURRENT_YEAR = 2020

paths = Paths()

# Population Data
file_path = paths.input_path + os.sep
FILE_NAME = r"population_summary.csv"

sfd = SimpleFleet(file_path + FILE_NAME)
sfd.load()
sfd.calc_age_forecast(START_YEAR, END_YEAR, CURRENT_YEAR);

In [ ]:
aml = AssetModelLoader(paths.demo_path + os.sep + "Asset Model - Pole - Timber.xlsx");
comp_data = aml.load(paths.demo_path + os.sep + "Asset Model - Pole - Timber.xlsx");
comp = Component.from_dict(comp_data["pole"]);
comp.fleet_data = sfd;  # TODO fix by creating asset class

In [4]:
t_end = 1000
n_iterations = 10
comp.mc_timeline(t_end=t_end, n_iterations=n_iterations)
comp.expected_risk_cost_df();

  0%|          | 0/10 [00:00<?, ?it/s]


TypeError: sim_timeline() got an unexpected keyword argument 't_end'

Existing function

In [5]:
import copy
import pandas as pd
def calc_df_task_forecast(comp, df_age_forecast, age_units="years"):
    """Create the task plot dataframe

    age_units - the units for the age in df_forecast age
    """

    # Scale the units to match the desired outputs
    df_age_forecast, __ = scale_units(df_age_forecast, age_units, comp.units)

    # Convert to floats for merging
    # TODO deep copy not needed anymore
    df_erc = copy.deepcopy(comp.df_erc)
    df_erc["time"] = df_erc["time"].astype(float)
    df_age_forecast["age"] = df_age_forecast["age"].astype(float)

    # Duplicate time so the closest age can be used
    df_age_forecast["time"] = df_age_forecast["age"]

    df_erc = pd.merge_asof(
        df_erc.sort_values("time"),
        df_age_forecast.sort_values("age")[["age", "time"]],
        on="time",
        tolerance=unit_ratio(age_units, comp.units),
    )

    # print(df_erc.sort_values("time"))
    # print(df_age_forecast.sort_values("age")[["age", "time"]])

    # Merge the population details
    df = pd.merge(
        df_erc.sort_values("time"),
        df_age_forecast.sort_values("age"),
        left_on="time",
        right_on="age",
    )

    # Calculated population outcomes
    df["pop_quantity"] = df["assets"] * df["quantity"]
    df["pop_cost"] = df["pop_quantity"] * df["cost"]

    # Regroup into a task forecast
    df = (
        df.groupby(by=["year", "task", "active"])[["pop_quantity", "pop_cost"]]
        .sum()
        .reset_index()
    )
    # print(df)
    # df_task = sort_df(df=df, column="task")

    return None #df

In [6]:
calc_df_task_forecast(comp=comp, df_age_forecast=sfd.df_age_forecast);

TypeError: 'NoneType' object is not subscriptable

In [7]:
comp.calc_pof_df(t_end=t_end);

ValueError: Values must be numeric: no strings, datetimes, objects, etc.

Alternate function - not going to be implemented

In [14]:
import scipy.stats as ss
import numpy as np
import pandas

def population_forecast(
    comp, 
    t_start=0, 
    t_end=100, 
    # df_pof=None, 
    df_age_forecast=None, 
    # input_units="months", 
    age_units="years"
):

    # df_pof, __ = scale_units(df_pof, age_units, input_units)

    comp.input_units = age_units
    pop = dict()

    for n in range(t_start, t_end):
        for fm, val in comp.fm.items():
            for task, var in val.tasks.items():
                t_start_cdf = n-1
                t_end_cdf = n
                
                X = np.linspace(t_start_cdf, t_end_cdf, t_end_cdf - t_start_cdf + 1)
                cdf = ss.weibull_min.cdf(X, val.untreated.beta, scale=val.untreated.alpha, loc=val.untreated.gamma)

                pop[str(n) + "_" + task] = {
                    "strategy": "no_maint", #val.untreated.strategy, #FIX THIS
                    "task": task,
                    "pof": cdf[0], #CHECK THIS,
                    "time": t_end_cdf,  #IS T_END IN MONTHS NOW?
                    "active": val.active
                }

    pop_df = pd.DataFrame.from_dict(pop, orient="index", columns=["strategy", "task", "pof", "time", "active"]) 
    
    # print(pop_df, df_age_forecast.head())

    df = pd.merge(
        pop_df.sort_values("time"),
        df_age_forecast.sort_values("age"),
        left_on="time",
        right_on="age",
    )

    # Calculated population outcomes
    df["pop_quantity"] = df["assets"] * df["pof"]
    # df["pop_cost"] = df["pop_quantity"] * df["cost"]

    # Regroup into a task forecast
    df = (
        df.groupby(by=["strategy", "year", "task", "active"])[["pop_quantity"]] #, "pop_cost"
        .sum()
        .reset_index()
    )

    # self.df_task = sort_df(df=df, column="task")

    # print(df)


    return None #df_task

In [15]:
population_forecast(comp=comp, df_age_forecast=sfd.df_age_forecast);
# df_pof=comp.df_pof,

In [10]:
# CURRENT
df = comp.calc_df_task_forecast(df_age_forecast=sfd.df_age_forecast, age_units="years");

df[df["pop_quantity"] > 5000];

TypeError: 'NoneType' object is not subscriptable

In [11]:
df_erc = comp.df_erc;

df_erc[df_erc["quantity"] > 0.1];

TypeError: 'NoneType' object is not subscriptable